In [ ]:
import pandas as pd
import numpy as np

def topsis_analysis(df, criteria_columns, weights, impacts):
    """
    Perform TOPSIS analysis on the dataset
    
    Parameters:
    - df: Input DataFrame
    - criteria_columns: List of columns to use for TOPSIS
    - weights: List of weights for each criterion
    - impacts: List of impact directions ('+' for maximize, '-' for minimize)
    
    Returns:
    - DataFrame with TOPSIS scores and ranks
    """
    # Create a copy of the dataframe
    data = df.copy()
    
    # Normalize the decision matrix
    normalized_matrix = data[criteria_columns].copy()
    for col in criteria_columns:
        # Normalize using vector normalization
        normalized_matrix[col] = normalized_matrix[col] / np.sqrt((normalized_matrix[col]**2).sum())
    
    # Apply weights
    weighted_matrix = normalized_matrix.copy()
    for i, col in enumerate(criteria_columns):
        weighted_matrix[col] *= weights[i]
    
    # Determine ideal best and worst solutions
    ideal_best = []
    ideal_worst = []
    for i, col in enumerate(criteria_columns):
        if impacts[i] == '+':
            ideal_best.append(weighted_matrix[col].max())
            ideal_worst.append(weighted_matrix[col].min())
        else:
            ideal_best.append(weighted_matrix[col].min())
            ideal_worst.append(weighted_matrix[col].max())

    # Calculate distances to ideal best and worst solutions
    distance_best = np.sqrt(((weighted_matrix[criteria_columns] - ideal_best)**2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_matrix[criteria_columns] - ideal_worst)**2).sum(axis=1))
    
    # Calculate TOPSIS score
    data['Topsis_Score'] = distance_worst / (distance_best + distance_worst)
    
    # Rank the alternatives
    data['Rank'] = data['Topsis_Score'].rank(method='dense', ascending=False)
    
    return data

# Read the dataset
df = pd.read_csv('data/something_new.csv')

# Get unique combinations of Size and Distance_x
combinations = df.groupby(['Size', 'Distance_x'])

# Store results
results = []

# Perform TOPSIS for each combination
for (size, distance), group in combinations:
    # Define criteria columns
    criteria_columns = ['carbon_emissions','insurance_cost','maintenance_cost','fuel_costs']
    
    # Define weights (equal weights in this case)
    weights = [0.5, 0.5, 0.5, 0.5]
    
    # Define impact directions (both to be minimized)
    impacts = ['-', '-', '-', '-']
    
    # Perform TOPSIS
    result = topsis_analysis(group, criteria_columns, weights, impacts)
    
    # Add size and distance info to results
    result['Size'] = size
    result['Distance'] = distance
    
    results.append(result)

# Combine all results
final_results = pd.concat(results)

# Save to CSV
final_results.to_csv('topsis_results.csv', index=False)

print("TOPSIS analysis completed. Results saved to topsis_results.csv")

TOPSIS analysis completed. Results saved to topsis_results.csv


In [16]:
df = pd.read_csv("topsis_results.csv")

In [ ]:
import pandas as pd
import numpy as np
import pulp

def optimize_fleet(
    initial_fleet,
    demand_forecast,
    vehicle_data,
    planning_horizon,
    discount_rate=0.05
):
    """
    Optimize fleet composition over multiple time periods.
    
    Parameters:
    - initial_fleet: DataFrame with columns [vehicle_type, quantity]
    - demand_forecast: Series or list of demand for each year
    - vehicle_data: DataFrame with columns [vehicle_type, year_available, purchase_cost, 
                   selling_value, operating_cost, yearly_range]
    - planning_horizon: Number of years to plan for
    - discount_rate: Annual discount rate for NPV calculations
    
    Returns:
    - DataFrame with optimal decisions for each year
    """
    # Initialize the optimization model
    model = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)
    
    # Extract all vehicle types that will be available during the planning horizon
    all_vehicle_types = vehicle_data['vehicle_type'].unique()
    years = range(1, planning_horizon + 1)
    
    # Create decision variables
    buy_vars = pulp.LpVariable.dicts("Buy", 
                                    ((i, t) for i in all_vehicle_types for t in years), 
                                    lowBound=0, 
                                    cat='Integer')
    sell_vars = pulp.LpVariable.dicts("Sell", 
                                     ((i, t) for i in all_vehicle_types for t in years), 
                                     lowBound=0, 
                                     cat='Integer')
    use_vars = pulp.LpVariable.dicts("Use", 
                                    ((i, t) for i in all_vehicle_types for t in years), 
                                    lowBound=0, 
                                    cat='Integer')
    
    # Initialize fleet state variables
    fleet = {}
    for i in all_vehicle_types:
        # Set initial fleet quantities
        initial_qty = initial_fleet.loc[initial_fleet['vehicle_type'] == i, 'quantity'].values
        fleet[(i, 0)] = initial_qty[0] if len(initial_qty) > 0 else 0
        
        # Create fleet state variables for future periods
        for t in years:
            fleet[(i, t)] = pulp.LpVariable(f"Fleet_{i}_{t}", lowBound=0, cat='Integer')
    
    # Objective function: Minimize total discounted cost
    total_cost = 0
    for t in years:
        discount_factor = 1 / (1 + discount_rate) ** (t-1)
        
        for i in all_vehicle_types:
            # Get vehicle data for this type and year
            vehicle_info = vehicle_data[
                (vehicle_data['vehicle_type'] == i) & 
                (vehicle_data['year_available'] <= t)
            ]
            
            if not vehicle_info.empty:
                latest_info = vehicle_info.iloc[-1]
                
                # Add costs to objective function
                total_cost += discount_factor * (
                    latest_info['purchase_cost'] * buy_vars[(i, t)] -
                    latest_info['selling_value'] * sell_vars[(i, t)] +
                    latest_info['operating_cost'] * use_vars[(i, t)]
                )
    
    model += total_cost
    
    # Constraints
    for t in years:
        # 1. Demand satisfaction constraint
        model += pulp.lpSum(
            vehicle_data.loc[vehicle_data['vehicle_type'] == i, 'yearly_range'].values[0] * 
            use_vars[(i, t)] for i in all_vehicle_types if not vehicle_data[
                (vehicle_data['vehicle_type'] == i) & 
                (vehicle_data['year_available'] <= t)
            ].empty
        ) >= demand_forecast[t-1], f"Demand_Satisfaction_Year_{t}"
        
        # 2. Fleet balance constraints
        for i in all_vehicle_types:
            model += fleet[(i, t)] == fleet[(i, t-1)] + buy_vars[(i, t)] - sell_vars[(i, t)], \
                    f"Fleet_Balance_{i}_{t}"
        
        # 3. Usage constraints
        for i in all_vehicle_types:
            model += use_vars[(i, t)] <= fleet[(i, t)], f"Usage_Constraint_{i}_{t}"
        
        # 4. Availability constraints
        for i in all_vehicle_types:
            is_available = not vehicle_data[
                (vehicle_data['vehicle_type'] == i) & 
                (vehicle_data['year_available'] <= t)
            ].empty
            
            if not is_available:
                model += buy_vars[(i, t)] == 0, f"Availability_Constraint_{i}_{t}"
            
        # 5. Selling constraints (can't sell more than what's in the fleet)
        for i in all_vehicle_types:
            model += sell_vars[(i, t)] <= fleet[(i, t-1)], f"Selling_Constraint_{i}_{t}"
    
    # Solve the model
    model.solve()
    
    # Extract results
    results = []
    for t in years:
        for i in all_vehicle_types:
            if pulp.value(buy_vars[(i, t)]) > 0 or \
               pulp.value(sell_vars[(i, t)]) > 0 or \
               pulp.value(use_vars[(i, t)]) > 0 or \
               pulp.value(fleet[(i, t)]) > 0:
                
                results.append({
                    'year': t,
                    'vehicle_type': i,
                    'buy': pulp.value(buy_vars[(i, t)]),
                    'sell': pulp.value(sell_vars[(i, t)]),
                    'use': pulp.value(use_vars[(i, t)]),
                    'fleet_size': pulp.value(fleet[(i, t)])
                })
    
    return pd.DataFrame(results)